<a href="https://colab.research.google.com/github/gabrielxbox/projeto-com-python/blob/master/algoritmo_de_classifica%C3%A7%C3%A3o_titanic_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv(“all/train.csv”)
df.head(10)

In [ ]:
df = df[["sobreviveu", "classe", "sexo", "idade", "parentes",
"dependentes", "tarifa", "cabine", "embarque"]]
df.head(10)

In [ ]:
labels = df.sobreviveu
labels.head(10)

In [ ]:
df.loc[pd.isna(df['classe']), 'classe'].shape

In [ ]:
df.loc[pd.isna(df['sexo']), 'sexo'].shape

In [ ]:
df.loc[pd.isna(df['idade']), 'idade'].shape

In [ ]:
indefinidos_idade = df.loc[pd.isna(df['idade']), 'idade'].shape[0]
total = df.idade.shape[0]
indefinidos_idade/total * 100

Os campos em que temos problemas são:
» idade (19.87%).
» cabine (77.10%).
» embarque (0.22%).
O campo cabine é o que mais compromete: 77,10% é muito! Em lugar de corrigir esses
dados é melhor assumir que simplesmente não possuímos essa informação e excluí-la
do nosso conjunto de dados.

In [ ]:
calcula_percentual_indefinido(df, 'parentes')

calcula_percentual_indefinido(df, 'dependentes')

calcula_percentual_indefinido(df, 'tarifa')

calcula_percentual_indefinido(df, 'cabine')

calcula_percentual_indefinido(df, 'embarque')

df = df.drop('cabine', axis=1)
df.head(10)

In [ ]:
df = df[np.isfinite(df['idade'])]
df.head(10)



In [ ]:
calcula_percentual_indefinido(df, 'idade')

In [ ]:
df = df[ ~pd.isna(df.embarque)]
calcula_percentual_indefinido(df, 'embarque')

In [ ]:
df.classe.unique()



def one_hot(categorias, valor):
 # Transforma as categorias num conjunto para eliminar duplicatas,
 # depois transforma em lista para obter índice
 categorias = list(set(categorias))

 # Obtém o índice do elemento atual
 indice = categorias.index(valor)

 # Cria um vetor com o número de coordenadas igual ao número
 de categorias
 vetor = np.zeros(len(categorias))

 # Modifica o índice correspondente a classe para 1.0 no ve
 tor de categorias
 vetor[indice] = 1.0

 # devolve o vetor one-hot resultante
 return vetor

 one_hot(df.classe, 1)

 one_hot(df.classe, 2)

 one_hot(df.classe, 3)

In [ ]:
df.sexo.unique()

In [ ]:
one_hot(df.sexo, 'male')

one_hot(df.sexo, 'female')

In [ ]:
df.embarque.unique()


In [ ]:
one_hot(df.embarque, 'S')

In [ ]:
one_hot(df.embarque, 'C')

In [ ]:
one_hot(df.embarque, 'Q')

In [ ]:
df.classe = [one_hot(df.classe, _) for _ in df.classe]

df.head(10)

In [ ]:
df.sexo = [one_hot(df.sexo, _) for _ in df.sexo]

df.head(10)

In [ ]:
df.embarque = [one_hot(df.embarque, _) for _ in df.embarque]

In [ ]:
df.head(10)

In [ ]:
df.sobreviveu.unique()

In [ ]:
one_hot(df.sobreviveu, 0)

In [ ]:
one_hot(df.sobreviveu, 1)

In [ ]:
df.sobreviveu = [one_hot(df.sobreviveu, _) for _ in
df.sobreviveu]
df.head(10)

In [ ]:
def normaliza_media(valores):
 media = np.mean(valores)
 minimo = np.min(valores)
 maximo = np.max(valores)

 return (valores - media)/(maximo-minimo), media, minimo, maximo

df.idade, media, minimo, maximo = normaliza_media(df.idade)
df.idade.head(10)

In [ ]:
df.parentes, media, minimo, maximo = normaliza_media(df.parentes)

In [ ]:
df.parentes.head(10)

In [ ]:
df.dependentes,media, minimo, maximo = normaliza_media(df.dependentes)
df.dependentes.head(10)

In [ ]:
df.tarifa, media, minimo, maximo = normaliza_media(df.tarifa)
df.tarifa.head(10)

In [ ]:
df.head(10)

In [ ]:
labels = df.sobreviveu.values

In [ ]:
df = df.drop(['sobreviveu'], axis=1)
df.head(10)

In [ ]:
def vetorizar(classe, sexo, idade, parentes, dependentes, tarifa, embarque):
 return classe.tolist() +\
 sexo.tolist() +\
 [idade] +\
 [parentes] +\
 [dependentes] + \
 [tarifa] +\
 embarque.tolist()

In [ ]:
entrada = [vetorizar(*_) for _ in df.values]

entrada[:5]

In [ ]:
np.savetxt("all/entrada_treino.csv", entrada, delimiter=',')

In [ ]:
np.savetxt("all/labels.csv", labels.tolist(), delimiter=',')

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from bokeh.plotting import figure, show, output_notebook

x = np.loadtxt("all/entrada_treino.csv", delimiter=',')
y = np.loadtxt("all/labels.csv", delimiter=',')

model = KNeighborsClassifier(n_neighbors=5)

In [ ]:
tamanho_treino = int(x.shape[0]*0.7)
tamanho_treino

In [ ]:
x_t = x[:tamanho_treino]
y_t = y[:tamanho_treino]

In [ ]:
x_v = x[tamanho_treino:]
y_v = y[tamanho_treino:]

In [ ]:
model.fit(x_t, y_t)

In [ ]:
KNeighborsClassifier(algorithm='auto', leaf_size=30,
metric='minkowski',
 metric_params=None, n_jobs=1, n_neighbors=5, p=2,
 weights='uniform')

In [ ]:
model.score(x_v, y_v)

In [ ]:
def modela_e_valida(k, x_t, y_t, x_v, y_v):
 # Cria o modelo parametrizado por k
 model = KNeighborsClassifier(n_neighbors=k)

 # Ajusta o modelo baseado no número de vizinhos
 model.fit(x_t, y_t)

 return k, model.score(x_v, y_v)

In [ ]:
performance_por_k = np.array([modela_e_valida(k, x_t, y_t, x_v,
y_v) for k in range(1, 100)])

In [ ]:
p = figure(title = "Performance do modelo de vizinhos mais próximos")

In [ ]:
p.xaxis.axis_label = 'Número de vizinhos'
p.yaxis.axis_label = 'Performance'
p.line(performance_por_k[:,0], performance_por_k[:,1])

In [ ]:
output_notebook()
show(p)

In [ ]:
p = figure(title = "Tempo de execução do modelo de vizinhos mais próximos")
p.xaxis.axis_label = 'Número de vizinhos'
p.yaxis.axis_label = 'Tempo de execução'
p.line(performance_por_k[:,0], performance_por_k[:,2])
output_notebook()
show(p)

In [ ]:
 acertos  = modela_e_valida(60, x_t, y_t, x_v, y_v)
acertos

In [ ]:
 acertos = modela_e_valida(80, x_t, y_t, x_v, y_v)
acertos